# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Updating registry at `~/.julia/registries/General`
┌ Warning: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Types.jl:1131
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


### Récupération des données

In [4]:
function readDynamic(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readDynamic (generic function with 1 method)

### Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [4]:
function TestsSondabilite_relaxlin(model2, x, varsbin, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) 
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objective_value(model2) >= BestProfit)
            Bestsol = value.(x)
            #BestProfit=value(benef)
            BestProfit=objective_value(model2)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_relaxlin (generic function with 1 method)

### Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [5]:

function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(varsshouldbebinary)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= 10^-5)
            var=varsshouldbebinary[i]
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable ?
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    set_lower_bound(var,1.0)
    set_upper_bound(var,1.0)

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listvars)>= 1)
        #go back to parent node
        var=pop!(listvars)
        theval=pop!(listvals)
        tmp=pop!(listnodes)
        set_lower_bound(var,0.0)
        set_upper_bound(var,1.0)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listvars)>= 1))
            var=pop!(listvars)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
            set_lower_bound(var,0.0) 
            set_upper_bound(var,1.0)
        end
        if theval==1.0
            set_lower_bound(var,0.0)
            set_upper_bound(var,0.0)
            push!(listvars,var)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, listnodes, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [72]:
function ResolutionDynamique(i, j, R, price, weight,indices_correspond,solution)
    # Cas de base
    if (j <= 0 || i == 0)   
        return 0,[v for v in solution]
    elseif (i == 1) 
        indices = findall(x -> x <= j, weight)
        if (indices == [])
            return 0,[v for v in solution]
        end
        choosen_prices = [price[indices[i]] for i in 1:length(indices)]
        choosen_indices_correspond = [indices_correspond[indices[i]] for i in 1:length(indices)]
        resultat = maximum(choosen_prices)
        rs = findall(x->x==resultat,choosen_prices)
        r = rs[1]
        newsolution = [v for v in solution]
        newsolution[choosen_indices_correspond[r]] = 1
        return resultat,newsolution
    else
        maxR = maximum(R)
        ks = findall(x -> x == maxR, R)
        k = ks[1]
        weight_choosen = weight[k]
        price_choosen = price[k]
        # mise a jour de R
        newR  = [v for v in R]
        deleteat!(newR, k)
        #mise à jour de price
        newprice  = [v for v in price]
        deleteat!(newprice, k)
        # mise à jour de weight
        newweight  = [ v for v in weight]
        deleteat!(newweight, k)
        #mise à jour de indices_correspond
        newindices_correspond = [v for v in indices_correspond]
        deleteat!(newindices_correspond, k)
        # Formule récursive
        ci1j,newsolution = ResolutionDynamique(i-1, j, newR, newprice, newweight,newindices_correspond,solution)
        if(weight_choosen <= j)
            ci1jk,newsolutionk = ResolutionDynamique(i-1, j-weight_choosen, newR, newprice, newweight,newindices_correspond,solution)
            if (ci1j <= ci1jk + price_choosen)
                newsolutionk[indices_correspond[k]] = 1
                return ci1jk + price_choosen,[v for v in newsolutionk]
            else
                return ci1j,[v for v in newsolution]
            end
        else
            return ci1j,[v for v in newsolution]
        end
    end
end

ResolutionDynamique (generic function with 2 methods)

### Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [76]:

function SolveDynamic(filename)

        price, weight, capacity = readDynamic(filename)
    
         
        # Liste contenant les ratios de tous les objets
        R = price./weight

        BestProfit = -1
        Bestsol = []
        
        j = capacity
        
        i = length(price)
        indices_correspond = [j for j in 1:i]
        solution = zeros(i)
            print("\nSolutionDynamique \n"); 
        
            BestProfit,solutionfinale = ResolutionDynamique(i, j, R, price, weight,indices_correspond,solution)

        return BestProfit,solutionfinale
   

end


SolveDynamic (generic function with 1 method)

### Affichage du résultat final

In [85]:
@time BestProfit,solution = SolveDynamic("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit,"\n******\n\nsolution optimale =",solution)


SolutionDynamique 
  0.000200 seconds (298 allocations: 14.922 KiB)

******

Optimal value = 65
******

solution optimale =[0.0, 1.0, 0.0, 1.0]


In [83]:
list_dossiers = readdir("instancesETU/KNAPnewformat")
deleteat!(list_dossiers,10)
println(list_dossiers)
for j in 1:length(list_dossiers)
    i = list_dossiers[j]
    list_fichiers = readdir("instancesETU/KNAPnewformat/"*i)
    println(i*" : ")
    println(list_fichiers)
    for k in 1:length(list_fichiers)
        f = list_fichiers[k]
        if ((findfirst("_20_",f) != nothing) || ((findfirst("_50_",f) != nothing) && ((findfirst("_5_",f) == nothing))))
            println(f)
            @time BestProfit, Bestsol = SolveDynamic("instancesETU/KNAPnewformat/"*i*"/"*f)
            println("\n******\n\nOptimal value = ", BestProfit,"\n******\n\nsolution optimale =",Bestsol)
        end
    end
end

["almost_strongly_correlated", "circle", "inverse_strongly_correlated", "multiple_strongly_correlated", "profit_ceiling", "similar_weights", "strongly_correlated", "strongly_correlated_span", "subset_sum", "uncorrelated", "uncorrelated_span", "weakly_correlated", "weakly_correlated_span"]
almost_strongly_correlated : 
["knapPI_5_10000_10000_1_-1484157.opb.txt", "knapPI_5_10000_10000_2_-2405704.opb.txt", "knapPI_5_10000_10000_3_-3195902.opb.txt", "knapPI_5_10000_10000_4_-4012574.opb.txt", "knapPI_5_10000_10000_5_-4687981.opb.txt", "knapPI_5_10000_1000_1_-145393.opb.txt", "knapPI_5_10000_1000_2_-238597.opb.txt", "knapPI_5_10000_1000_3_-320932.opb.txt", "knapPI_5_10000_1000_4_-396116.opb.txt", "knapPI_5_10000_1000_5_-466744.opb.txt", "knapPI_5_1000_10000_1_-146390.opb.txt", "knapPI_5_1000_10000_2_-243637.opb.txt", "knapPI_5_1000_10000_3_-322069.opb.txt", "knapPI_5_1000_10000_4_-393112.opb.txt", "knapPI_5_1000_10000_5_-468478.opb.txt", "knapPI_5_1000_1000_1_-14202.opb.txt", "knapPI_5_1000_

InterruptException: [91mInterruptException:[39m